<a href="https://colab.research.google.com/github/mohamedalaaaz/testpytroch/blob/main/altrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Dummy dataset
X = torch.randn(100, 10)
y = torch.randint(0, 2, (100,))

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# Simple model
model = nn.Sequential(
    nn.Linear(10, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    for batch_X, batch_y in loader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.6644
Epoch 2, Loss: 0.6592
Epoch 3, Loss: 0.7210
Epoch 4, Loss: 0.8651
Epoch 5, Loss: 0.7366
Epoch 6, Loss: 0.7525
Epoch 7, Loss: 0.6257
Epoch 8, Loss: 0.5100
Epoch 9, Loss: 0.7081
Epoch 10, Loss: 0.8305


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

# Sample dataset (replace with your own)
texts = ["What is AI?", "Define gravity.", "Who is the president?"]
labels = [0, 1, 0]  # 0 = general, 1 = science, etc.

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom Dataset
class QADataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {**{k: v[idx] for k, v in self.encodings.items()}, 'labels': self.labels[idx]}

    def __len__(self):
        return len(self.labels)

dataset = QADataset(texts, labels)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):
    for batch in loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Loss = 0.7796
Epoch 2: Loss = 0.6243
Epoch 3: Loss = 0.4245


In [6]:
# After model prediction
user_question = "What is the capital of France?"
model_answer = "Paris"

# Ask user for feedback
feedback = input(f"Model answer: {model_answer}\nWas this correct? (y/n): ")

# Save feedback
with open("feedback_log.txt", "a") as f:
    f.write(f"{user_question}\t{model_answer}\t{feedback}\n")


Model answer: Paris
Was this correct? (y/n): n


In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import torch.optim as optim

# Sample dataset — replace with your own
questions = [
    "What is the capital of France?",
    "Explain Newton's second law.",
    "Who painted the Mona Lisa?",
    "What is 2 + 2?"
]
labels = [0, 1, 0, 1]  # 0 = general, 1 = science (example labels)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Custom Dataset
class QuestionDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = QuestionDataset(questions, labels)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):
    for batch in loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# Switch to eval mode for feedback
model.eval()

# Predict + Collect Feedback
print("\n🔎 Now testing and collecting feedback...")
for question in questions:
    inputs = tokenizer(question, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted = torch.argmax(probs).item()

    label_name = "general" if predicted == 0 else "science"
    print(f"\nQ: {question}")
    print(f"Model Prediction: {label_name}")

    # Ask for feedback
    feedback = input("Is this correct? (y/n): ").strip().lower()

    # Save feedback
    with open("feedback_log.txt", "a") as f:
        f.write(f"{question}\t{label_name}\t{feedback}\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.7184
Epoch 2, Loss: 0.5911
Epoch 3, Loss: 0.5360

🔎 Now testing and collecting feedback...

Q: What is the capital of France?
Model Prediction: general
Is this correct? (y/n): pairs

Q: Explain Newton's second law.
Model Prediction: science
Is this correct? (y/n): n

Q: Who painted the Mona Lisa?
Model Prediction: general
Is this correct? (y/n): n

Q: What is 2 + 2?
Model Prediction: science
Is this correct? (y/n): y


In [8]:
pip install transformers torch

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model (small one for demo – replace with Mistral or LLaMA if you have GPU)
model_name = "tiiuae/falcon-7b-instruct"  # or "mistralai/Mistral-7B-Instruct" if you have resources
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()

print("🧠 Chatbot is ready! Type 'exit' to quit.\n")

while True:
    user_input = input("🧑 You: ")
    if user_input.lower() == "exit":
        break

    # Create prompt
    prompt = f"<|user|>: {user_input}\n<|assistant|>:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    bot_reply = response.split("<|assistant|>:")[-1].strip()
    print(f"🤖 Bot: {bot_reply}")

    # Collect feedback
    feedback = input("Was this helpful? (y/n or correct answer): ").strip()

    # Save feedback
    with open("chatbot_feedback.txt", "a", encoding="utf-8") as f:
        f.write(f"Q: {user_input}\nA: {bot_reply}\nFeedback: {feedback}\n\n")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Same chatbot + feedback logic as before...


In [ ]:
pip install gradio


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
model_name = "mistralai/Mistral-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
model.eval()

# Chat function
def chat_with_feedback(user_input, history=[]):
    prompt = f"<|user|>: {user_input}\n<|assistant|>:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    bot_reply = response.split("<|assistant|>:")[-1].strip()

    # Add to history
    history.append((user_input, bot_reply))
    return history, history

# Feedback saving
def save_feedback(history, feedback_text):
    last_q, last_a = history[-1]
    with open("gradio_feedback.txt", "a", encoding="utf-8") as f:
        f.write(f"Q: {last_q}\nA: {last_a}\nFeedback: {feedback_text}\n\n")
    return "✅ Feedback saved!"

# Gradio UI
chatbot_ui = gr.Chatbot()
interface = gr.Interface(
    fn=chat_with_feedback,
    inputs=gr.Textbox(placeholder="Ask a question...", label="Your Question"),
    outputs=[chatbot_ui, "state"],
    live=True
)

feedback_box = gr.Textbox(label="Feedback (optional)")
feedback_button = gr.Button("Submit Feedback")
feedback_status = gr.Text(label="")

feedback_button.click(fn=save_feedback, inputs=["state", feedback_box], outputs=feedback_status)

interface.launch()


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ✅ Load model (use Falcon for lighter RAM use; Mistral for better quality)
model_name = "tiiuae/falcon-7b-instruct"  # Replace with "mistralai/Mistral-7B-Instruct" if you have VRAM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# 🧠 Chat function
def chat_with_feedback(user_input, history=[]):
    prompt = f"<|user|>: {user_input}\n<|assistant|>:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    bot_reply = response.split("<|assistant|>:")[-1].strip()

    history.append((user_input, bot_reply))
    return history, history

# 📝 Save feedback
def save_feedback(history, feedback_text):
    if not history:
        return "❌ No conversation to save."
    last_q, last_a = history[-1]
    with open("feedback_log.txt", "a", encoding="utf-8") as f:
        f.write(f"Q: {last_q}\nA: {last_a}\nFeedback: {feedback_text}\n\n")
    return "✅ Feedback saved!"

# 🔧 Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Mistral Chatbot with Feedback")
    chatbot = gr.Chatbot()
    state = gr.State([])
    with gr.Row():
        msg = gr.Textbox(placeholder="Ask a question...", label="Your Question")
        send_btn = gr.Button("Send")
    with gr.Row():
        feedback = gr.Textbox(placeholder="Optional feedback...", label="Your Feedback")
        feedback_btn = gr.Button("Submit Feedback")
        status = gr.Text()

    send_btn.click(chat_with_feedback, inputs=[msg, state], outputs=[chatbot, state])
    feedback_btn.click(save_feedback, inputs=[state, feedback], outputs=status)

demo.launch()


In [ ]:
import json

def parse_feedback_log(log_file, output_file):
    dataset = []
    with open(log_file, 'r', encoding='utf-8') as f:
        lines = f.read().strip().split("\n\n")
        for entry in lines:
            lines = entry.strip().split("\n")
            if len(lines) != 3:
                continue  # skip malformed
            q = lines[0].replace("Q: ", "").strip()
            a = lines[1].replace("A: ", "").strip()
            feedback = lines[2].replace("Feedback: ", "").strip()

            # Optional: skip poor feedback
            if feedback.lower() in ['n', 'no', 'bad', 'wrong']:
                continue

            item = {
                "instruction": q,
                "input": "",
                "output": a
            }
            dataset.append(item)

    # Save as JSONL
    with open(output_file, 'w', encoding='utf-8') as out_f:
        for item in dataset:
            out_f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"✅ Converted {len(dataset)} entries to {output_file}")

# Run it
parse_feedback_log("feedback_log.txt", "fine_tune_dataset.jsonl")


In [ ]:
pip install transformers datasets trl accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

# 🧾 Load your dataset (JSONL)
dataset = load_dataset("json", data_files="fine_tune_dataset.jsonl", split="train")

# 🔧 Model + tokenizer (same as you used in chatbot)
model_name = "mistralai/Mistral-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# ✅ Tokenizer config
tokenizer.pad_token = tokenizer.eos_token

# ⚙️ Training arguments
training_args = TrainingArguments(
    output_dir="./mistral-finetuned-feedback",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-5,
    fp16=True,
    save_total_limit=2,
)

# 🏋️ Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=lambda example: {
        "text": f"<|user|>: {example['instruction']}\n<|assistant|>: {example['output']}"
    }
)

# 🚀 Start training
trainer.train()
